In [1]:
import sys
sys.path.append('C:/Users/tjker/Desktop/Research/Projects/lit_review/lit_review')
import utils

sys.path.append('C:/Users/tjker/Desktop/Research/Projects/lit_review/configs')
from create_chunks_config import config

kg = utils.load_kg(config)

In [2]:
from langchain_neo4j import Neo4jVector
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
# from langchain.chains import RetrievalQAWithSourcesChain

from dotenv import load_dotenv
import textwrap

load_dotenv('C:/Users/tjker/Desktop/Research/Projects/lit_review/.env', override=True)

llm = HuggingFaceEndpoint(model=config['model']['model_id'])

chunk_vector = Neo4jVector.from_existing_index(
    HuggingFaceEmbeddings(model_name=config['embedding']['model_id']),
    graph=kg,
    index_name='paper_chunks',
    embedding_node_property='textEmbedding',
    text_node_property='text',
)

# Powerful Querying capability
A query like the following:
```
MATCH (p:Paper)-[:HAS_CHUNK]->(c:Chunk)
WHERE p.publication_date >= $min_date AND p.author = $desired_author
WITH c, p, vector.similarity.cosine(c.embedding, $embedding) AS score
ORDER BY score DESC LIMIT $k
RETURN c, score, {title: p.title, url: p.url} AS metadata
```

Allows me to first filter the vector database that I look at and then perform RAG. This is far more targeted than looking at all chunks in the database.

# QA with sources retriever

In [3]:
def display_response(response):
    print("Question:")
    print(response.get("question", ""))
    print("\nAnswer:")
    print(response.get("answer", ""))
    print("\nSources:")
    # If your sources key contains a string or list, print it nicely
    sources = response.get("sources", [])
    if isinstance(sources, list):
        for i, src in enumerate(sources, start=1):
            print(f"  [{i}] {src}")
    else:
        print(sources)
    print("\nDetailed Source Documents:")
    docs = response.get("source_documents", [])
    for i, doc in enumerate(docs, start=1):
        # Customize as needed; here we assume each doc is a Document object with metadata
        title = doc.metadata.get("source", "Unknown Source")
        text_snippet = doc.page_content.replace("\n", " ") + "..."
        print(f"Source {i}: {title}")
        print(f"Snippet: {text_snippet}\n")

In [6]:
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

custom_query = """
MATCH (c:Chunk)
WITH DISTINCT c, vector.similarity.cosine(c.textEmbedding, $embedding) AS score
ORDER BY score DESC LIMIT $k
RETURN c.text AS text, score, {source: c.source, chunkId: c.chunkId} AS metadata
"""

filtered_chunk_vector = Neo4jVector.from_existing_index(
    HuggingFaceEmbeddings(model_name=config['embedding']['model_id']),
    graph=kg,
    index_name='paper_chunks',
    embedding_node_property='textEmbedding',
    text_node_property='text',
    retrieval_query=custom_query,
)

# retriever = filtered_chunk_vector.as_retriever()
retriever_k = filtered_chunk_vector.as_retriever(search_kwargs={
        "k": 3
    })

qa_chain = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,
    retriever=retriever_k,
    return_source_documents=True,  # so you get citations back
)

In [ ]:
question = "What are some ways that you can leverage the structure of a latent space to influence generation?"
response = qa_chain.invoke({"question": question})
# Then call your display function:
display_response(response)


In [ ]:
print(textwrap.fill(response['answer'], 60))

In [ ]:
question = "What are some ways that you can leverage the structure of a latent space to influence generation?"
response = chunk_retriever.invoke({"query": question})

print(textwrap.fill(response['result'], 60))

In [36]:
# question = "How can we uncover the underlying structure of the latent space in GANs and Diffusion Models?"
# result = chunk_vector.similarity_search(question, k=3)
# for doc in result:
#     print(doc.metadata["chunkId"], "-", doc.page_content)

In [46]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever_0
)

In [ ]:
question = "What are some ways that the structure of a latent space can be leveraged to influence the generation process?"
response = chain.invoke({"question": question},
        return_only_outputs=True,)

# print(textwrap.fill(response['result'], 60))
response

In [ ]:
import textwrap
def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain.invoke({"question": question},
        return_only_outputs=True,)
    print(textwrap.fill(response['answer'], 60))
    
prettychain('who wrote Self-Guided Diffusion Models?')

In [ ]:
chain.invoke({"question": 'who wrote Self-Guided Diffusion Models?'})

In [ ]:
chain.invoke({"question": 'who wrote DiGress: Discrete Denoising diffusion for graph generation?'})

# Old graph_rag.ipynb

In [ ]:
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():     
        outputs = model(**inputs) 
    return outputs.last_hidden_state.mean(dim=1).squeeze(0).tolist()

question = "Transformer architecture"
# question_embedding = model.encode(question)
question_embedding = compute_embedding(question)

kg.query("""
    CALL db.index.vector.queryNodes(
        'abstract_embeddings', 
        $top_k, 
        $question_embedding
        ) YIELD node AS paper, score
    RETURN paper.title, paper.abstract, score
    """, 
    params={"top_k":5,
            "question_embedding": question_embedding
            })

In [ ]:
title = 'Latent Space Editing in Transformer-Based Flow Matching'
kg.query("""
    CALL db.index.fulltext.queryNodes('paperTitleIndex', $title)     
    YIELD node, score
    RETURN node.paperId, score
    LIMIT 1
    """, params={'title': title}
)